In [1]:
import pandas as pd
import requests
import time
import json
from io import StringIO
from Bio import SeqIO

In [2]:
bcell_url = "https://query-api.iedb.org/bcell_search"
bcell_search_params = {
    'source_organism_iri_search': 'cs.{"NCBITaxon:2"}',
    'host_organism_iri_search': 'cs.{"NCBITaxon:9606"}',
    'disease_iri_search': 'cs.{"DOID:0050117"}',
    'qualitative_measure': 'neq.Negative',
    'limit': 1000,
    'offset': 0,
    'order': 'bcell_iri'
}

bcell_results = requests.get(bcell_url, params=bcell_search_params)
bcell_search_params['offset'] += len(bcell_results.json())
print(bcell_search_params['offset'])
bcell_data = bcell_results.json()

while (bcell_results.json() != []):
    time.sleep(2)
    bcell_results = requests.get(bcell_url, params=bcell_search_params)
    if (bcell_results.json() != []):
        bcell_search_params['offset'] += len(bcell_results.json())
        print(bcell_search_params['offset'])
        bcell_data.extend(bcell_results.json())
    else:
        break

1000
2000
2798


In [3]:
bcell_df = pd.DataFrame(bcell_data)
bcell_df

,bcell_id,bcell_iri,structure_id,structure_iri,linear_sequence,structure_type,structure_description,curated_source_antigen,reference_id,reference_iri,...,non_peptidic_molecule_iri,non_peptidic_molecule_name,r_object_source_molecule_iri_search,r_object_source_molecule_iri,r_object_source_molecule_name,r_object_source_organism_iri_search,r_object_source_organism_iri,r_object_source_organism_name,e_related_object_type,host_mhc_types_present
0,1041040,IEDB_ASSAY:1041040,41948,IEDB_EPITOPE:41948,MLGNAPSVVPNTTLGM,Linear peptide,MLGNAPSVVPNTTLGM,"{'accession': 'A43589', 'name': 'mtp40 protein...",1000620,IEDB_REFERENCE:1000620,...,None,None,None,None,None,None,None,None,None,None
1,1041055,IEDB_ASSAY:1041055,66693,IEDB_EPITOPE:66693,TTLGMHCGSFGSAPSNG,Linear peptide,TTLGMHCGSFGSAPSNG,"{'accession': 'A43589', 'name': 'mtp40 protein...",1000620,IEDB_REFERENCE:1000620,...,None,None,None,None,None,None,None,None,None,None
2,1041060,IEDB_ASSAY:1041060,72741,IEDB_EPITOPE:72741,WLKLGLVEFGGVAKLNAEVMS,Linear peptide,WLKLGLVEFGGVAKLNAEVMS,"{'accession': 'A43589', 'name': 'mtp40 protein...",1000620,IEDB_REFERENCE:1000620,...,None,None,None,None,None,None,None,None,None,None
3,1041067,IEDB_ASSAY:1041067,41952,IEDB_EPITOPE:41952,MLGTGTPNRARINFNC,Linear peptide,MLGTGTPNRARINFNC,"{'accession': 'A43589', 'name': 'mtp40 protein...",1000620,IEDB_REFERENCE:1000620,...,None,None,None,None,None,None,None,None,None,None
4,1041069,IEDB_ASSAY:1041069,27635,IEDB_EPITOPE:27635,INFNCEVWSNVSETISGPRLY,Linear peptide,INFNCEVWSNVSETISGPRLY,"{'accession': 'A43589', 'name': 'mtp40 protein...",1000620,IEDB_REFERENCE:1000620,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2793,9067,IEDB_ASSAY:9067,41071,IEDB_EPITOPE:41071,MAKVNIKPLEDKILV,Linear peptide,MAKVNIKPLEDKILV,"{'accession': 'P09621.3', 'name': '10 kDa chap...",833,IEDB_REFERENCE:833,...,None,None,None,None,None,None,None,None,None,None
2794,9080,IEDB_ASSAY:9080,72294,IEDB_EPITOPE:72294,WDEDGEKRIPLDVAE,Linear peptide,WDEDGEKRIPLDVAE,"{'accession': 'P09621.3', 'name': '10 kDa chap...",833,IEDB_REFERENCE:833,...,None,None,None,None,None,None,None,None,None,None
2795,9082,IEDB_ASSAY:9082,35316,IEDB_EPITOPE:35316,LDVAEGDTVIYSKYG,Linear peptide,LDVAEGDTVIYSKYG,"{'accession': 'P09621.3', 'name': '10 kDa chap...",833,IEDB_REFERENCE:833,...,None,None,None,None,None,None,None,None,None,None
2796,9176063,IEDB_ASSAY:9176063,1125229,IEDB_EPITOPE:1125229,SEKQMPSVVNENAVTPEK,Linear peptide,SEKQMPSVVNENAVTPEK,"{'accession': 'WP_106459654.1', 'name': 'LPXTG...",1037215,IEDB_REFERENCE:1037215,...,None,None,None,None,None,None,None,None,None,None


In [4]:
tcell_url = "https://query-api.iedb.org/tcell_search"
tcell_search_params = {
    'source_organism_iri_search': 'cs.{"NCBITaxon:2"}',
    'host_organism_iri_search': 'cs.{"NCBITaxon:9606"}',
    'disease_iri_search': 'cs.{"DOID:0050117"}',
    'qualitative_measure': 'neq.Negative',
    'limit':1000,
    'offset': 0,
    'order': 'tcell_iri'
}
tcell_data=[]
tcell_results = requests.get(tcell_url, params=tcell_search_params)
tcell_search_params['offset'] += len(tcell_results.json())
print(tcell_search_params['offset'])
tcell_data = tcell_results.json()

while (tcell_results.json() != []):
    time.sleep(2)
    tcell_results = requests.get(tcell_url, params=tcell_search_params)
    if (tcell_results.json() != []):
        tcell_search_params['offset'] += len(tcell_results.json())
        print(tcell_search_params['offset'])
        tcell_data.extend(tcell_results.json())
    else:
        break

1000
2000
3000
4000
4898


In [5]:
tcell_df = pd.DataFrame(tcell_data)
tcell_df

,tcell_id,tcell_iri,structure_id,structure_iri,linear_sequence,structure_type,structure_description,curated_source_antigen,reference_id,reference_iri,...,non_peptidic_molecule_iri,non_peptidic_molecule_name,r_object_source_molecule_iri_search,r_object_source_molecule_iri,r_object_source_molecule_name,r_object_source_organism_iri_search,r_object_source_organism_iri,r_object_source_organism_name,e_related_object_type,host_mhc_types_present
0,10035,IEDB_ASSAY:10035,55139,IEDB_EPITOPE:55139,RPEAVLQHARTLAKI,Linear peptide,RPEAVLQHARTLAKI,"{'accession': 'P45487.1', 'name': '8-amino-7-o...",890,IEDB_REFERENCE:890,...,None,None,None,None,None,None,None,None,None,None
1,10037,IEDB_ASSAY:10037,41499,IEDB_EPITOPE:41499,MFGCMNYSTRVTLAD,Linear peptide,MFGCMNYSTRVTLAD,"{'accession': 'P35901.1', 'name': 'Protein rec...",890,IEDB_REFERENCE:890,...,None,None,None,None,None,None,None,None,None,None
2,10039,IEDB_ASSAY:10039,41499,IEDB_EPITOPE:41499,MFGCMNYSTRVTLAD,Linear peptide,MFGCMNYSTRVTLAD,"{'accession': 'P35901.1', 'name': 'Protein rec...",890,IEDB_REFERENCE:890,...,None,None,None,None,None,None,None,None,None,None
3,10041,IEDB_ASSAY:10041,20811,IEDB_EPITOPE:20811,GLDSIISSASASLLT,Linear peptide,GLDSIISSASASLLT,"{'accession': 'Q07297.1', 'name': 'Serine-rich...",890,IEDB_REFERENCE:890,...,None,None,None,None,None,None,None,None,None,None
4,10043,IEDB_ASSAY:10043,20811,IEDB_EPITOPE:20811,GLDSIISSASASLLT,Linear peptide,GLDSIISSASASLLT,"{'accession': 'Q07297.1', 'name': 'Serine-rich...",890,IEDB_REFERENCE:890,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,9815,IEDB_ASSAY:9815,69572,IEDB_EPITOPE:69572,VLKSYVLEGTLTAEK,Linear peptide,VLKSYVLEGTLTAEK,"{'accession': 'P14013.1', 'name': 'Outer surfa...",626,IEDB_REFERENCE:626,...,None,None,None,None,None,None,None,None,None,None
4894,9897,IEDB_ASSAY:9897,69572,IEDB_EPITOPE:69572,VLKSYVLEGTLTAEK,Linear peptide,VLKSYVLEGTLTAEK,"{'accession': 'P14013.1', 'name': 'Outer surfa...",626,IEDB_REFERENCE:626,...,None,None,None,None,None,None,None,None,None,None
4895,9898,IEDB_ASSAY:9898,69572,IEDB_EPITOPE:69572,VLKSYVLEGTLTAEK,Linear peptide,VLKSYVLEGTLTAEK,"{'accession': 'P14013.1', 'name': 'Outer surfa...",626,IEDB_REFERENCE:626,...,None,None,None,None,None,None,None,None,None,None
4896,9913,IEDB_ASSAY:9913,69569,IEDB_EPITOPE:69569,VLKNFTLEGKVANDK,Linear peptide,VLKNFTLEGKVANDK,"{'accession': 'P0A3N7.1', 'name': 'Outer surfa...",626,IEDB_REFERENCE:626,...,None,None,None,None,None,None,None,None,None,None


In [20]:
peptide_bcell_df = bcell_df[bcell_df['structure_type'].str.contains("peptide")].reset_index(drop=True)
peptide_tcell_df = tcell_df[tcell_df['structure_type'].str.contains("peptide")].reset_index(drop=True)
peptide_df = pd.concat([peptide_bcell_df, peptide_tcell_df], axis=0).reset_index(drop=True)
peptide_df = peptide_df[~peptide_df['parent_source_antigen_iri'].isna()].reset_index(drop=True)
antigen_ids = list(set(peptide_df['parent_source_antigen_iri'].tolist()))
for i in range(len(antigen_ids)):
    antigen_ids[i] = antigen_ids[i].split(':')[1]
    
with open("antigen_ids.txt", "w") as file:
    file.write(",".join(antigen_ids))

# Using UniProt ID-Mapping To Map antigen_ids



In [58]:
ID_mapping_post_url = "https://rest.uniprot.org/idmapping/run"
ID_mapping_post_params = {
    "from": "UniProtKB_AC-ID",
    "to": "UniProtKB",
    "ids": antigen_ids
}

jobId = requests.post(ID_mapping_post_url, data=ID_mapping_post_params).json()['jobId']
jobId

'z6xtoOlDWe'

In [59]:
ID_mapping_status_url = f"https://rest.uniprot.org/idmapping/status/{jobId}"
status = requests.get(ID_mapping_status_url)
print(json.dumps(status.json(), indent=4))
while status.json()['jobStatus'] in ["RUNNING", "NEW", "QUEUED"]:
    print("Fetching...")
    time.sleep(5)
    status = requests.get(ID_mapping_status_url)
    if 'jobStatus' not in status.json():
        break

print(json.dumps(status.json(), indent=4))

{
    "results": [
        {
            "from": "O50391",
            "to": {
                "entryType": "UniProtKB unreviewed (TrEMBL)",
                "primaryAccession": "O50391",
                "secondaryAccessions": [
                    "F2GEA8",
                    "I6Y3A9",
                    "Q7D5M6"
                ],
                "uniProtkbId": "O50391_MYCTU",
                "entryAudit": {
                    "firstPublicDate": "1998-06-01",
                    "lastAnnotationUpdateDate": "2025-10-08",
                    "lastSequenceUpdateDate": "1998-06-01",
                    "entryVersion": 142,
                    "sequenceVersion": 1
                },
                "annotationScore": 1.0,
                "organism": {
                    "scientificName": "Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv)",
                    "taxonId": 83332,
                    "evidences": [
                        {
                            "evidenceCode": 

KeyError: 'jobStatus'

In [31]:
ID_mapping_results_url = f"https://rest.uniprot.org/idmapping/uniprotkb/results/stream/{jobId}"
ID_mapping_results_params = {
    "query": "taxonomy_id:2 AND reviewed:true AND fragment:false",
    "format": "fasta"
}

result = requests.get(ID_mapping_results_url, params=ID_mapping_results_params)
antigen_sequences = result.text
print(antigen_sequences)
with open("antigenic_sequences.fasta", "w") as file:
    file.write(antigen_sequences)

In [82]:
ID_mapping_results_url = f"https://rest.uniprot.org/idmapping/uniprotkb/results/stream/{jobId}"
ID_mapping_results_params = {
    "query": "taxonomy_id:2 AND reviewed:true AND fragment:false",
    "format": "tsv",
    "fields": "accession,id,protein_name,gene_names,organism_name,organism_id,length,mass,cc_subcellular_location,ft_signal,ft_helix,ft_strand,ft_turn,reviewed"

result = requests.get(ID_mapping_results_url, params=ID_mapping_results_params)
tsv_antigen = pd.read_csv(StringIO(result.text), sep='\t')
tsv_antigen

,From,Entry,Entry Name,Protein names,Gene Names,Organism,Organism (ID),Length,Mass,Subcellular location [CC],Signal peptide,Helix,Beta strand,Turn,Reviewed
0,A0PSI5,A0PSI5,MASZ_MYCUA,Malate synthase G (EC 2.3.3.9),glcB MUL_3055,Mycobacterium ulcerans (strain Agy99),362242,731,79479,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
1,A1JIP3,A1JIP3,CH60_YERE8,Chaperonin GroEL (EC 5.6.1.7) (60 kDa chaperon...,groEL groL YE0354,Yersinia enterocolitica serotype O:8 / biotype...,393305,550,57513,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
2,A1JNN2,A1JNN2,CLPP_YERE8,ATP-dependent Clp protease proteolytic subunit...,clpP YE3134,Yersinia enterocolitica serotype O:8 / biotype...,393305,207,23256,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
3,A1JUB7,A1JUB7,YADA_YERE8,Adhesin YadA (Protein Yop1) (Trimeric autotran...,yadA yopA YEP0066,Yersinia enterocolitica serotype O:8 / biotype...,393305,422,44139,SUBCELLULAR LOCATION: Cell surface {ECO:000026...,"SIGNAL 1..25; /evidence=""ECO:0000255""","HELIX 333..360; /evidence=""ECO:0007829|PDB:2LME""","STRAND 368..380; /evidence=""ECO:0007829|PDB:2L...","TURN 408..410; /evidence=""ECO:0007829|PDB:2LME""",reviewed
4,A5CDL9,A5CDL9,CH60_ORITB,Chaperonin GroEL (EC 5.6.1.7) (60 kDa chaperon...,groEL groL OTBS_0917,Orientia tsutsugamushi (strain Boryong) (Ricke...,357244,554,59715,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,Q9Z5I7,Q9Z5I7,SYP_MYCLE,Proline--tRNA ligase (EC 6.1.1.15) (Prolyl-tRN...,proS ML1553 MLCB596.17c,Mycobacterium leprae (strain TN),272631,480,53585,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
490,Q9Z6U5,Q9Z6U5,PMP21_CHLPN,Probable outer membrane protein pmp21 (Polymor...,pmp21 CPn_0963 CP_0897 CpB1000,Chlamydia pneumoniae (Chlamydophila pneumoniae),83558,1609,170866,"SUBCELLULAR LOCATION: Secreted, cell wall. Cel...","SIGNAL 1..30; /evidence=""ECO:0000255""",NaN,NaN,NaN,reviewed
491,Q9Z7A6,Q9Z7A6,ENO_CHLPN,Enolase (EC 4.2.1.11) (2-phospho-D-glycerate h...,eno CPn_0800 CP_1071 CpB0829,Chlamydia pneumoniae (Chlamydophila pneumoniae),83558,428,46032,SUBCELLULAR LOCATION: Cytoplasm {ECO:0000255|H...,NaN,NaN,NaN,NaN,reviewed
492,Q9Z899,Q9Z899,PMP6_CHLPN,Probable outer membrane protein pmp6 (Polymorp...,pmp6 CPn_0444 CP_0309 CpB0460,Chlamydia pneumoniae (Chlamydophila pneumoniae),83558,1276,132127,"SUBCELLULAR LOCATION: Secreted, cell wall. Cel...","SIGNAL 1..23; /evidence=""ECO:0000255""",NaN,NaN,NaN,reviewed


In [57]:
non_antigens_url = "https://rest.uniprot.org/uniprotkb/stream"
non_antigens_search_params = {
    "query": "taxonomy_id:2 AND reviewed:true AND fragment:false",
    "format": "fasta"
}

result = requests.get(non_antigens_url, params=non_antigens_search_params)
non_antigen_sequences = result.text
print(non_antigen_sequences)
with open("non_antigenic_sequences.fasta", "w") as file:
    file.write(non_antigen_sequences)

Columns retrieved: ['Entry', 'Entry Name', 'Protein names', 'Gene Names', 'Length', 'Mass', 'Binding site', 'Subunit structure', 'Subcellular location [CC]', 'Signal peptide', 'Beta strand', 'Helix', 'Turn']


,Entry,Entry Name,Protein names,Gene Names,Length,Mass,Binding site,Subunit structure,Subcellular location [CC],Signal peptide,Beta strand,Helix,Turn
0,P12345,AATM_RABIT,"Aspartate aminotransferase, mitochondrial (mAs...",GOT2,430,47409,"BINDING 65; /ligand=""substrate""; /evidence=""EC...",SUBUNIT: Homodimer. {ECO:0000250}.,SUBCELLULAR LOCATION: Mitochondrion matrix {EC...,NaN,NaN,NaN,NaN


In [11]:
non_antigens_url = "https://rest.uniprot.org/uniprotkb/stream"
non_antigens_search_params = {
    "query": "taxonomy_id:2 AND reviewed:true AND fragment:false",
    "format": "tsv",
    "fields": "accession,id,protein_name,gene_names,organism_name,organism_id,length,mass,cc_subcellular_location,ft_signal,ft_helix,ft_strand,ft_turn,reviewed"
}

result = requests.get(non_antigens_url, params=non_antigens_search_params)
tsv_non_antigen = pd.read_csv(StringIO(result.text), sep='\t')
tsv_non_antigen

In [12]:
# for i in url_ids:
#     with open("url_ids.txt", "w")as file:
#         file.write(i)

In [13]:
# url_ids_txt = ",".join(url_ids)
# with open("url_ids.txt", "w") as file:
#     for i in range(100000):
#         file.write(","+url_ids[i])

In [14]:
# down_ids=[]
# for record in SeqIO.parse("/Users/gnanasandeep/Desktop/uniprotkb_taxonomy_id_2_AND_reviewed_tr_2025_12_07.fasta", "fasta"):
#     down_ids.append(record.id)

In [15]:
# # down_ids = down_ids['Entry'].tolist()
# len(list(set(down_ids+url_ids)))